In [1]:
import numpy as np
import functions as func
import matplotlib.pyplot as plt
import pdb, glob, mpld3, time
import mirpyidl as idl
from tqdm import tqdm
from astropy.io import fits, ascii
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from mpld3 import plugins

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 8)
mpld3.enable_notebook()

**File name directory example:  
bd +60 1753 all ch1 data: /data1/phot_cal/spitzer/bd601753/r*/ch1/bcd/*_bcd.fits**

In [77]:
# fnames = np.sort(glob.glob(raw_input('path to bcd files:')))
fnames = np.sort(glob.glob('../bd_aor_test/r20966144/ch1/bcd/*_bcd.fits'))
len(fnames)

20

**Next two cells generates flux data with no systematics removed**

In [41]:
#Provide proper sky coordinates in hms
# '18 02 30.7410086899 +58 37 38.157415821' for HD 165459
# '17 24 52.2772360943 +60 25 50.780790994' for BD +60 1753

# sky = SkyCoord(raw_input('Target coordinates in hms:'), unit=(u.hourangle, u.deg))
sky = SkyCoord('17 24 52.2772360943 +60 25 50.780790994', unit=(u.hourangle, u.deg))

In [78]:
#Play with filenames to see what portion of the name you want in data table
a,b = 44, 64
fnames[5][a:b]

'20966144_0002_0001_4'

In [79]:
#Issues list
crd_conversion = []
centroiding    = []
bad_cen_guess  = []
not_in_fov     = []

data = Table(names = ('File#','ACenX', 'ACenY', 'FCenX', 'FCenY', 'Time[MJD]', 'Raw_Flux', 'Bkg_Flux', 'Res_Flux'), 
             dtype = ('S25', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'))


for fn in tqdm(fnames):
    
    hdu    = fits.open(fn)
    header = hdu[0].header
    image  = hdu[0].data
    hdu.close()

    fnum = fn[a:b]
    Time = header['MJD_OBS']

    try:
        w = WCS(header)
        pix = sky.to_pixel(w)
    except:
        crd_conversion.append(fnum)
        continue

    if (pix[0]>0) & (pix[0]<256) & (pix[1]>0) & (pix[1]<256):

        try:
            cenX, cenY = func.gen_center_g2d(pix[0], pix[1], 7, 5, 4, 4, 0, image)
        except:
            centroiding.append(fnum)
            continue
        
        if (np.abs(cenX - pix[0]) <= 2) & (np.abs(cenY-pix[1]) <= 2):
            
            try:
                # Extracting raw flux
                raw_flux, src_ap = func.photometry(image, [cenX], [cenY], rad = 10)

                # Extrating a mean background flux
                bkg, bkg_ap = func.photometry(image, [cenX], [cenY], shape = 'CircAnn', r_in = 12, r_out = 20)
                bkg_mean = bkg/bkg_ap.area()
                bkg_flux = bkg_mean*src_ap.area()

                # Subtracting background
                res_flux  = raw_flux - bkg_flux

                data.add_row([fnum, pix[0], pix[1], cenX, cenY, Time, raw_flux, bkg_flux, res_flux])
            
            except:
                continue
            
        else:
            bad_cen_guess.append(fnum)
            
    else:
        not_in_fov.append(fnum)

100%|██████████| 20/20 [00:03<00:00,  5.58it/s]


**Run next cell if you want to save the data generated from previous cell  
Uncomment the first 3 lines if you want to save the file names that are causing problems  
Change file names if you don't want to overwrite**

In [80]:
#np.save('crd_conversion_issue_bd_ch1.npy', np.array(crd_conversion))
#np.save('centroiding_issue_bd_ch1.npy', np.array(centroiding))
#np.save('bad_cen_guess_bd_ch1.npy', np.array(bad_cen_guess))

# n = '../data_for_idl/' + raw_input('filename:')
n = '../data_for_idl/bd_r' + str(header['AORKEY']) + '_bcd.csv'
ascii.write(data, n, delimiter = ',', overwrite = True)
print n

../data_for_idl/bd_r20966144_bcd.csv


**Remove systematics using IDL code  
Systematics removed through this process: Array location dependent correction, pixel phase correction, callibration factor**

```idl
data  = read_csv('data_for_idl/xyz.csv') ;Use proper file path
cenX  = data.field4                      ;Usually column 4 is the fitted cenX column of my generated table
cenY  = data.field5                      ;Be careful about which column you are using
oFlux = data.field9                      ;We use the residual flux column for observed flux

;Now to calculate corrected flux
;Make sure to have irac_aphot_corr_cryo.pro in the same folder
;You might need to comple it using .compile irac_aphot_corr_cryo.pro command 

cFlux = IRAC_APHOT_CORR_CRYO(oFlux, cenX, cenY, 1)
fname = 'idl_results/xyz.csv'
write_csv, fname, cFlux, header = ['cFlux']

;Useful commands:
;help, data  (works with any array)
;print, data 
```

**Use last systematics removed flux to remove another systematics: Aperture correction**

In [81]:
# corr_flux = ascii.read(raw_input('corrected flux file path:'))  
corr_flux = ascii.read('../idl_results/bd_r20966144_bcd_res.csv')  

In [12]:
# Find the aperture correction factor from the following link:
# https://irsa.ipac.caltech.edu/data/SPITZER/docs/irac/iracinstrumenthandbook/27/
#from the table at the end of that link, select your value accoring to your aperture size and channel

ap_corr = 1.000

In [84]:
#write the refined flux to the data table in a new column

"""
#Uncomment this section if you want to load a previously generated data file
data = ascii.read(raw_input('data table file path:'))   
#Otherwise, data generated in this notebook will be used
"""

data['Refined_Flux'] = np.array(corr_flux).astype('Float64')*ap_corr

**Go from MJy/Sr to Jy**

In [82]:
pixLen  = 1.221 #arcsec
pixArea = pixLen**2 #arcsec^2

# 1 rad = 206265"
# 1 sr = 1 rad^2 = (206265**2) arcsec^2
pixArea = pixArea/(206265**2) #Steradian

apRadius = 10 #pixel length
apArea   = np.pi*(apRadius**2) #pixel area
apArea   = apArea*pixArea #Steradian
apArea

1.1008549975617078e-08

In [85]:
data['Raw_Flux'] *= (pixArea*(10**9)) #Multiply by apArea to get to MJy and by 10^9 to get to mJy
data['Bkg_Flux'] *= (pixArea*(10**9))
data['Res_Flux'] *= (pixArea*(10**9))
data['Refined_Flux'] *= (pixArea*(10**9))

**Run next cell if you want to save refined data with correct units to a csv file  
To activate the cell, select it, press esc and then y  
Change file names if you don't want to overwrite**

In [235]:
# ascii.write(data, raw_input('file path:'), overwrite = True)
ascii.write(data, '../bd_aor_test/r65217280_bcd.csv', overwrite = True)

file path:../bd_aor_test/r65464064_bcd.csv


**Optional Analysis
To activate the cell, select it, press esc and then y**

In [86]:
mean  = np.mean(data['Refined_Flux'])
stdev = np.std(data['Refined_Flux'])
mean, stdev, (stdev/mean)*100

(38.99355643687791, 0.40161428099055035, 1.0299503756234099)